In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("All libraries imported successfully!")


All libraries imported successfully!


In [ ]:
try:
    # If you have multiple sheets, specify sheet_name
    df = pd.read_excel('Glioma-clinic-TCGA .xlsx', sheet_name=0)
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print("Dataset file not found. Creating sample data for demonstration...")
    # Creating sample data structure based on brain cancer research
    np.random.seed(42)
    n_samples = 500
    
    # Sample data generation
    df = pd.DataFrame({
        'Patient_ID': [f'TCGA-{i:03d}' for i in range(n_samples)],
        'Age': np.random.normal(55, 15, n_samples).astype(int),
        'Gender': np.random.choice(['Male', 'Female'], n_samples),
        'Tumor_Type': np.random.choice(['Oligodendroglioma', 'Oligoastrocytoma', 'Astrocytoma', 'Glioblastoma'], 
                                     n_samples, p=[0.2, 0.15, 0.25, 0.4]),
        'Grade': np.random.choice(['Grade II', 'Grade III', 'Grade IV'], n_samples, p=[0.3, 0.3, 0.4]),
        'Survival_Months': np.random.exponential(24, n_samples),
        'IDH1_Mutation': np.random.choice(['Mutant', 'Wild-type'], n_samples, p=[0.6, 0.4]),
        'MGMT_Methylation': np.random.choice(['Methylated', 'Unmethylated'], n_samples, p=[0.45, 0.55]),
        'Protein_1': np.random.normal(2.5, 1.2, n_samples),
        'Protein_2': np.random.normal(1.8, 0.8, n_samples),
        'Protein_3': np.random.normal(3.2, 1.5, n_samples),
        'Protein_4': np.random.normal(2.1, 0.9, n_samples),
        'Protein_5': np.random.normal(1.5, 0.7, n_samples),
        'Ki67_Index': np.random.normal(15, 8, n_samples),
        'Tumor_Size_cm': np.random.normal(4.2, 1.8, n_samples),
        'Necrosis_Present': np.random.choice(['Yes', 'No'], n_samples, p=[0.35, 0.65])
    })

print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'brain_cancer_data.csv'